In [1]:
# This notebook writes data to the Neo4j graph.

In [1]:
from neo4j import GraphDatabase

In [2]:
uri = "bolt://localhost:7687"
user = "neo4j"
password = "password"

In [3]:
driver = GraphDatabase.driver(uri, auth=(user, password))

In [4]:
session = driver.session()

In [ ]:
# loading phone nodes into graph; csv needs to be in graph's import folder
session.run(
    '''LOAD CSV WITH HEADERS FROM 'file:///phone_nodes.csv' AS row
    WITH row
    CALL {
        WITH row
        CREATE (:Identifier {
            identifier: row.identifier,
            type: row.type
            })
    };''')

In [11]:
# create an index for phone identifiers
session.run(
    '''CREATE INDEX identifier_range_index FOR (i:Identifier) ON (i.identifier, i.type);''')

In [6]:
# loading phone relationships into graph
session.run(
    '''LOAD CSV WITH HEADERS FROM 'file:///phone_relationships_v1.csv' AS row
    WITH row
    CALL {
        WITH row
        MATCH (a:Identifier), (b:Identifier) WHERE a.identifier = row.source and b.identifier = row.target
        MERGE (a)-[:CONTACT {first_seen:row.first_seen, last_seen:row.last_seen, times:row.times, collection:row.collection}]->(b)
    };''')

In [6]:
# loading person nodes into graph; csv needs to be in graph's import folder
session.run(
    '''LOAD CSV WITH HEADERS FROM 'file:///users.csv' AS row
    WITH row
    CALL {
        WITH row
        CREATE (:Person {
            first_name: row.first_name,
            last_name: row.last_name,
            full_name: row.full_name,
            birthday: row.birthday
            })
    };''')

In [8]:
session.run('''
CREATE INDEX person_index FOR (n:Person) ON (n.birthday, n.full_name)''')

In [13]:
# loading user-phone relationships into graph
session.run(
    '''LOAD CSV WITH HEADERS FROM 'file:///phones_users_v1.csv' AS row
    WITH row
    CALL {
        WITH row
        MATCH (p:Person), (i:Identifier) WHERE p.full_name = row.full_name AND p.birthday = row.birthday
        AND i.identifier = row.identifier AND i.type = row.type
        MERGE (p)-[:USER_OF {collection: row.collection}]->(i)
    };''')

In [6]:
# update comms relation with new comms
session.run(
    '''LOAD CSV WITH HEADERS FROM 'file:///new_comms_update.csv' AS row
    WITH row
    CALL {
    WITH row
    MATCH (a:Identifier),(b:Identifier) WHERE a.identifier = row.source AND b.identifier = row.target
    MERGE (a)-[c1:CONTACT]->(b)
    ON CREATE SET c1.last_seen = row.last_seen,c1.times = row.times,c1.first_seen=row.first_seen,c1.collection=row.collection
    ON MATCH SET c1.last_seen = row.last_seen,c1.times = row.times
};''')